<h1 style="text-align: center;">DPDQ HW</h1>
<h2 style="text-align: center;">Name : Siddhesh Maheshwari</h2>
<h2 style="text-align: center;">MDS202347</h2>

# Data Quality Analysis on Adult Dataset
#### Using Great Expectations

##### **Importing necessary lib and dataset**

In [1]:
import warnings
warnings.filterwarnings( "ignore",    message=".*should_run_async.*",category=DeprecationWarning)
warnings.filterwarnings("ignore")

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
from IPython.core.display import display, HTML
for dirname, _, filenames in os.walk(f'E:\Sem 4\DPDQ\HW1\DPDQ\adult'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [3]:
# !pip install great_expectations

## Data loading and basic data cleaning like data type changing 

In [4]:
column_names = [
    'age', 'workclass', "fnlwgt",'education', 'education-num', 'marital-status', 
    'occupation', 'relationship', 'race', 'sex', 'capital-gain', 
    'capital-loss', 'hours-per-week', 'native-country', 'income'
]
input
adult_data = pd.read_csv(r"E:\Sem 4\DPDQ\HW1\adult\adult.data", names=column_names)
adult_test = pd.read_csv(r"E:\Sem 4\DPDQ\HW1\adult\adult.test", names=column_names)
adult_test.drop(index=0,inplace=True)

adult_data_full = pd.concat([adult_data, adult_test], axis=0)

adult_data_full.index = adult_data_full.index.astype(str)

int_columns = ['age', 'fnlwgt', 'education-num', 'capital-gain', 'capital-loss', 'hours-per-week']
for col in int_columns:
    adult_data_full[col] = adult_data_full[col].astype(int)

In [5]:
adult_data_full.sample(5)

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
19628,36,State-gov,76767,Some-college,10,Divorced,Exec-managerial,Not-in-family,White,Female,0,0,39,United-States,<=50K
7252,59,?,259673,Some-college,10,Married-civ-spouse,?,Husband,Other,Male,0,0,40,Puerto-Rico,<=50K.
13826,26,Private,56929,Bachelors,13,Never-married,Exec-managerial,Not-in-family,Black,Male,0,0,50,?,<=50K
13811,43,State-gov,598995,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,3103,0,40,United-States,>50K.
4918,41,Self-emp-not-inc,443508,HS-grad,9,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,50,United-States,<=50K


In [6]:
adult_data_full.info()

<class 'pandas.core.frame.DataFrame'>
Index: 48842 entries, 0 to 16281
Data columns (total 15 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   age             48842 non-null  int32 
 1   workclass       48842 non-null  object
 2   fnlwgt          48842 non-null  int32 
 3   education       48842 non-null  object
 4   education-num   48842 non-null  int32 
 5   marital-status  48842 non-null  object
 6   occupation      48842 non-null  object
 7   relationship    48842 non-null  object
 8   race            48842 non-null  object
 9   sex             48842 non-null  object
 10  capital-gain    48842 non-null  int32 
 11  capital-loss    48842 non-null  int32 
 12  hours-per-week  48842 non-null  int32 
 13  native-country  48842 non-null  object
 14  income          48842 non-null  object
dtypes: int32(6), object(9)
memory usage: 4.8+ MB


In [7]:
string_columns = adult_data_full.select_dtypes(include=['object']).columns

for col in string_columns:
    adult_data_full[col] = adult_data_full[col].str.strip()

In [8]:
string_columns

Index(['workclass', 'education', 'marital-status', 'occupation',
       'relationship', 'race', 'sex', 'native-country', 'income'],
      dtype='object')

In [9]:
adult_data_full.shape

(48842, 15)

In [10]:
adult_data_full.sample(5)

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
5573,33,Private,164748,Bachelors,13,Married-civ-spouse,Transport-moving,Husband,White,Male,0,0,40,United-States,<=50K
22974,32,Private,239659,Some-college,10,Separated,Machine-op-inspct,Unmarried,Black,Female,0,0,70,United-States,<=50K
8888,29,Private,226295,HS-grad,9,Never-married,Handlers-cleaners,Not-in-family,White,Male,0,0,36,United-States,<=50K.
5126,42,Private,173938,Bachelors,13,Married-civ-spouse,Other-service,Husband,White,Male,0,0,40,?,<=50K.
980,49,Private,123207,HS-grad,9,Never-married,Adm-clerical,Not-in-family,White,Female,0,0,44,United-States,<=50K


# **Great Expectation**

In [11]:
import great_expectations as gx
print(gx.__version__)

1.0.0a1


In [12]:
from great_expectations import get_context  

# Initialize the DataContext
context = get_context()

# Now you can create your dataset with the active context
adult_data_gx = gx.dataset.PandasDataset(adult_data_full)

In [13]:
adult_data_gx.expect_column_values_to_be_between("age",min_value=1, max_value=100)

{
  "success": true,
  "result": {
    "element_count": 48842,
    "missing_count": 0,
    "missing_percent": 0.0,
    "unexpected_count": 0,
    "unexpected_percent": 0.0,
    "unexpected_percent_total": 0.0,
    "unexpected_percent_nonmissing": 0.0,
    "partial_unexpected_list": []
  },
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}

In [14]:
adult_data_gx.expect_column_values_to_be_in_set("workclass", ('Private', 'Self-emp-not-inc', 'Self-emp-inc', 'Federal-gov', 'Local-gov', 'State-gov', 'Without-pay', 'Never-worked',"?"))

{
  "success": true,
  "result": {
    "element_count": 48842,
    "missing_count": 0,
    "missing_percent": 0.0,
    "unexpected_count": 0,
    "unexpected_percent": 0.0,
    "unexpected_percent_total": 0.0,
    "unexpected_percent_nonmissing": 0.0,
    "partial_unexpected_list": []
  },
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}

In [15]:
adult_data_gx.expect_column_values_to_be_in_set("education", ('Bachelors', 'Some-college', '11th', 'HS-grad', 'Prof-school', 'Assoc-acdm', 'Assoc-voc', '9th', '7th-8th', '12th', 'Masters', '1st-4th', '10th', 'Doctorate', '5th-6th', 'Preschool'))


{
  "success": true,
  "result": {
    "element_count": 48842,
    "missing_count": 0,
    "missing_percent": 0.0,
    "unexpected_count": 0,
    "unexpected_percent": 0.0,
    "unexpected_percent_total": 0.0,
    "unexpected_percent_nonmissing": 0.0,
    "partial_unexpected_list": []
  },
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}

In [16]:
adult_data_gx.expect_column_values_to_be_in_set("marital-status", ('Married-civ-spouse', 'Divorced', 'Never-married', 'Separated', 'Widowed', 'Married-spouse-absent', 'Married-AF-spouse'))

{
  "success": true,
  "result": {
    "element_count": 48842,
    "missing_count": 0,
    "missing_percent": 0.0,
    "unexpected_count": 0,
    "unexpected_percent": 0.0,
    "unexpected_percent_total": 0.0,
    "unexpected_percent_nonmissing": 0.0,
    "partial_unexpected_list": []
  },
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}

In [17]:
adult_data_gx.expect_column_values_to_be_in_set("occupation", ("?",'Tech-support', 'Craft-repair', 'Other-service', 'Sales', 'Exec-managerial', 'Prof-specialty', 'Handlers-cleaners', 'Machine-op-inspct', 'Adm-clerical', 'Farming-fishing', 'Transport-moving', 'Priv-house-serv', 'Protective-serv', 'Armed-Forces'))

{
  "success": true,
  "result": {
    "element_count": 48842,
    "missing_count": 0,
    "missing_percent": 0.0,
    "unexpected_count": 0,
    "unexpected_percent": 0.0,
    "unexpected_percent_total": 0.0,
    "unexpected_percent_nonmissing": 0.0,
    "partial_unexpected_list": []
  },
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}

In [18]:
adult_data_gx.expect_column_values_to_be_in_set("relationship", ('Wife', 'Own-child', 'Husband', 'Not-in-family', 'Other-relative', 'Unmarried'))

{
  "success": true,
  "result": {
    "element_count": 48842,
    "missing_count": 0,
    "missing_percent": 0.0,
    "unexpected_count": 0,
    "unexpected_percent": 0.0,
    "unexpected_percent_total": 0.0,
    "unexpected_percent_nonmissing": 0.0,
    "partial_unexpected_list": []
  },
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}

In [19]:
adult_data_gx.expect_column_values_to_be_in_set("race", ('White', 'Asian-Pac-Islander', 'Amer-Indian-Eskimo', 'Other', 'Black'))

{
  "success": true,
  "result": {
    "element_count": 48842,
    "missing_count": 0,
    "missing_percent": 0.0,
    "unexpected_count": 0,
    "unexpected_percent": 0.0,
    "unexpected_percent_total": 0.0,
    "unexpected_percent_nonmissing": 0.0,
    "partial_unexpected_list": []
  },
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}

In [20]:
adult_data_gx.expect_column_values_to_be_in_set("sex", ('Female', 'Male'))

{
  "success": true,
  "result": {
    "element_count": 48842,
    "missing_count": 0,
    "missing_percent": 0.0,
    "unexpected_count": 0,
    "unexpected_percent": 0.0,
    "unexpected_percent_total": 0.0,
    "unexpected_percent_nonmissing": 0.0,
    "partial_unexpected_list": []
  },
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}

In [21]:
adult_data_gx.expect_column_values_to_be_in_set("native-country", ("?",'United-States', 'Cambodia', 'England', 'Puerto-Rico', 'Canada', 'Germany', 'Outlying-US(Guam-USVI-etc)', 'India', 'Japan', 'Greece', 'South', 'China', 'Cuba', 'Iran', 'Honduras', 'Philippines', 'Italy', 'Poland', 'Jamaica', 'Vietnam', 'Mexico', 'Portugal', 'Ireland', 'France', 'Dominican-Republic', 'Laos', 'Ecuador', 'Taiwan', 'Haiti', 'Columbia', 'Hungary', 'Guatemala', 'Nicaragua', 'Scotland', 'Thailand', 'Yugoslavia', 'El-Salvador', 'Trinadad&Tobago', 'Peru', 'Hong', 'Holand-Netherlands'))

{
  "success": true,
  "result": {
    "element_count": 48842,
    "missing_count": 0,
    "missing_percent": 0.0,
    "unexpected_count": 0,
    "unexpected_percent": 0.0,
    "unexpected_percent_total": 0.0,
    "unexpected_percent_nonmissing": 0.0,
    "partial_unexpected_list": []
  },
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}

In [22]:
adult_data_gx.expect_column_values_to_be_in_set("income", ('<=50K', '>50K'))

{
  "success": false,
  "result": {
    "element_count": 48842,
    "missing_count": 0,
    "missing_percent": 0.0,
    "unexpected_count": 16281,
    "unexpected_percent": 33.33401580606854,
    "unexpected_percent_total": 33.33401580606854,
    "unexpected_percent_nonmissing": 33.33401580606854,
    "partial_unexpected_list": [
      "<=50K.",
      "<=50K.",
      ">50K.",
      ">50K.",
      "<=50K.",
      "<=50K.",
      "<=50K.",
      ">50K.",
      "<=50K.",
      "<=50K.",
      ">50K.",
      "<=50K.",
      "<=50K.",
      "<=50K.",
      ">50K.",
      ">50K.",
      "<=50K.",
      "<=50K.",
      "<=50K.",
      ">50K."
    ]
  },
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}

In [38]:
adult_data_gx["income"] = adult_data_gx["income"].str.strip().str.replace(".", "", regex=False)
adult_data_full["income"] = adult_data_full["income"].str.strip().str.replace(".", "", regex=False)

In [39]:
adult_data_gx.expect_column_values_to_be_in_set("income", ('<=50K', '>50K'))

{
  "success": true,
  "result": {
    "element_count": 48842,
    "missing_count": 0,
    "missing_percent": 0.0,
    "unexpected_count": 0,
    "unexpected_percent": 0.0,
    "unexpected_percent_total": 0.0,
    "unexpected_percent_nonmissing": 0.0,
    "partial_unexpected_list": []
  },
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}

# Dry Running the expectations

In [46]:
adult_data_gx = gx.dataset.PandasDataset(adult_data_full)

# === Manual Expectations ===

# Age: Between 17 and 90
adult_data_gx.expect_column_values_to_be_between("age", min_value=17, max_value=90)
adult_data_gx.expect_column_values_to_not_be_null("age")

# Workclass: One of the known categories (no missing)
adult_data_gx.expect_column_values_to_be_in_set("workclass", [
    "Private", "Self-emp-not-inc", "Self-emp-inc", "Federal-gov", "Local-gov", "State-gov", "Without-pay", "Never-worked","?"
])
adult_data_gx.expect_column_values_to_not_be_null("workclass")

# fnlwgt: Positive integers
adult_data_gx.expect_column_values_to_be_between("fnlwgt", min_value=1)
adult_data_gx.expect_column_values_to_not_be_null("fnlwgt")

# Education
adult_data_gx.expect_column_values_to_be_in_set("education", [
    "Bachelors", "HS-grad", "11th", "Masters", "9th", "Some-college", "Assoc-acdm",
    "Assoc-voc", "7th-8th", "Doctorate", "Prof-school", "5th-6th", "1st-4th", "10th",
    "Preschool", "12th"
])
adult_data_gx.expect_column_values_to_not_be_null("education")

# education-num: 1 to 16
adult_data_gx.expect_column_values_to_be_between("education-num", min_value=1, max_value=16)

# marital-status
adult_data_gx.expect_column_values_to_be_in_set("marital-status", [
    "Married-civ-spouse", "Divorced", "Never-married", "Separated", "Widowed", "Married-spouse-absent", "Married-AF-spouse"
])
adult_data_gx.expect_column_values_to_not_be_null("marital-status")

# occupation (might need to handle missing values separately)
adult_data_gx.expect_column_values_to_not_be_null("occupation")

# relationship
adult_data_gx.expect_column_values_to_be_in_set("relationship", [
    "Wife", "Own-child", "Husband", "Not-in-family", "Other-relative", "Unmarried"
])
adult_data_gx.expect_column_values_to_not_be_null("relationship")

# race
adult_data_gx.expect_column_values_to_be_in_set("race", [
    "White", "Asian-Pac-Islander", "Amer-Indian-Eskimo", "Other", "Black"
])

# sex
adult_data_gx.expect_column_values_to_be_in_set("sex", ["Male", "Female"])

# capital-gain and capital-loss: >= 0
adult_data_gx.expect_column_values_to_be_between("capital-gain", min_value=0)
adult_data_gx.expect_column_values_to_be_between("capital-loss", min_value=0)

# hours-per-week: reasonable range
adult_data_gx.expect_column_values_to_be_between("hours-per-week", min_value=1, max_value=100)

# native-country
adult_data_gx.expect_column_values_to_not_be_null("native-country")

# income: clean categories
adult_data_gx.expect_column_values_to_be_in_set("income", ["<=50K", ">50K"])

# === Run the validation ===
result = adult_data_gx.validate()

# Print result summary
from pprint import pprint
print("\n Manual Validation Summary:")
pprint(result["statistics"])



 Manual Validation Summary:
{'evaluated_expectations': 21,
 'success_percent': 100.0,
 'successful_expectations': 21,
 'unsuccessful_expectations': 0}
